In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd
import numpy as np
from src.walkforward import walk_forward_backtest

from strategies.continuous_allocation import ContinuousAllocationStrategy
from strategies.MLDynamicStrategy import MLProbabilisticStrategy


In [ ]:
# ================================================================
# Step 1: Define Parameters and Load Data
# ================================================================
data_path = r'C:\Users\epoch_bpjmdqk\Documents\Code\data\processed\stock_and_macro.csv'
try:
    data = pd.read_csv(data_path, index_col='Date', parse_dates=True)
except FileNotFoundError:
    print(f"Error: Data file not found at {data_path}. Please check your path.")
    sys.exit()

target_ticker = "WMT"
backtest_start_date = "2021-01-01"

# We use the entire dataset here; the walk-forward function will handle the splits
# internally, starting its test periods from the specified date.
print(f"✅ Loaded data from {data.index.min().strftime('%Y-%m-%d')} to {data.index.max().strftime('%Y-%m-%d')}.")

In [ ]:
# ================================================================
# Step 2: Run the Walk-Forward Backtest
# ================================================================
print("\n--- Starting Walk-Forward Backtest ---")
try:
    stats_list, full_equity, metrics_summary = walk_forward_backtest(
        data=data,
        target_ticker=target_ticker,
        trading_strategy=ContinuousAllocationStrategy, # Pass the class reference here
        window=10,
        threshold=0.005,
        backtest_start_date=backtest_start_date,
        n_splits=5
    )

    # ================================================================
    # Step 3: Display Results
    # ================================================================
    print("\n--- Walk-Forward Summary Statistics (Aggregated) ---")
    print(metrics_summary)

    # Print per-fold results for a detailed view
    for i, stats in enumerate(stats_list, 1):
        print(f"\n--- Fold {i} Results ---")
        print(stats)
        # We can also plot individual fold results if needed, e.g.:
        # stats._equity_curve.plot(label=f"Fold {i}")

    # ================================================================
    # Step 4: Create a Buy-and-Hold Benchmark for Comparison
    # ================================================================
    starting_balance = 10000.0
    buy_and_hold_data = data.loc[backtest_start_date:].copy()
    buy_and_hold_shares = int(starting_balance / buy_and_hold_data.iloc[0][f'Close_{target_ticker}'])
    buy_and_hold_initial_cash = starting_balance - (buy_and_hold_shares * buy_and_hold_data.iloc[0][f'Close_{target_ticker}'])
    buy_and_hold_balance = buy_and_hold_data[f'Close_{target_ticker}'] * buy_and_hold_shares + buy_and_hold_initial_cash

    # ================================================================
    # Step 5: Visualize the Final Equity Curves
    # ================================================================
    plt.figure(figsize=(12, 8))
    plt.style.use('seaborn-v0_8-darkgrid')
    
    # Plot the walk-forward strategy's equity curve
    full_equity.plot(label='Walk-Forward Strategy', color='red', linewidth=2)
    
    # Plot the buy-and-hold benchmark
    buy_and_hold_balance.plot(label='Buy and Hold', color='skyblue', linewidth=2)
    
    plt.title(f'Walk-Forward Backtest: {target_ticker} Strategy vs. Buy and Hold', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Account Balance ($)', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"\nAn error occurred during the walk-forward backtest: {e}")
